In [84]:
import sympy as smp
from sympy.integrals.integrals import integrate

#### Goal:

Given a Riemannian Metric defined on a manifold, compute as many *intrinsic* properties, meaning properties that can be derived by only knowning the metric and the respective coordinate system. Intuitively, "intrinsic" means that a blind observer could measure these properties by only moving along the manifold, without being aware of the ambient space in which the manifold might be embedded.

#### Define the Riemannian Metric

The metric is a symmetric bilinear form that when restricted on the tangent space of the manifold defines an inner product.
 $$g: T_{p}\mathcal{M} \times T_{p} \mathcal{M} \rightarrow \mathbb{R}$$Essentially, it is a $n \times n$ symmetric matrix, where $n$ is the dimension of the manifold (or, equivalently, of the tangent space).

In [85]:
# Example: Standard metric of the 2-sphere in spherical coordinates (theta, phi)
# R = smp.Symbol('R')
# theta = smp.Symbol('theta')
# phi = smp.Symbol('phi')
# g = smp.Matrix([[R**2, 0], [0, R**2 * (smp.sin(theta))**2]])

In [86]:
# g

#### First Fundamental Form Coefficient

It is an equivalent way to encode the metric. For the 2-sphere and the metric defined above, the first fundamental form is givem by the following formula:

$$ds^2 = E d\theta \otimes d\theta + 2F d\phi \otimes d\theta + G d\phi \otimes d\phi $$

,where $E = g_{00} = R^2$, $F = g_{01} = g_{10} = 0$ and $G = g_{11} = R^2 \sin^2\theta$. And so, for the 2-sphere: $$ds^2 = R^2
 d\theta \otimes d\theta + R^2 \sin^2\theta d\phi \otimes d\phi$$

In [87]:
def first_fundamental_form(metric):
    E, F, G = metric.row(0)[0], metric.row(0)[1], metric.row(1)[1]
    return E, F, G

#### Compute the inverse metric

It is the inverse by the usual linear algebra sense and it is denoted by $g_{ij}^{-1} = g^{ij}$

In [88]:
def inverse_metric(metric):
    assert metric.det() != 0, 'The defined metric is not invertible.'
    return metric.inv()

#### Compute the Christoffel Symbols of the first and/or second kind

Geometrically, the Christoffel Symbols of the second kind tell us how the basis vectors change from point to point on a manifold. If our manifold has dimension $n$, then we can define $n^3$ *functions* (they are **not** tensors), $\Gamma_{ij}^{k}$, as: $$\Gamma_{ij}^{k} = \frac{\partial \textbf{e}_{i}}{\partial x^{j}} \cdot \textbf{e}^{k}$$

In local coordinates they can be computed by using the metric as follows $$\displaystyle \Gamma_{ij}^{k} = \frac{1}{2} \sum_{m=1}^{n} g^{km} \Big( \frac{\partial g_{jm}}{\partial x^{i}} + \frac{\partial g_{mi}}{\partial x^{j}} - \frac{\partial g_{ij}}{\partial x^{k}}       \Big)$$

In [89]:
def christoffel_symbols(metric, coordinates, kind='second'):
    assert kind == 'first' or kind == 'second', 'Invalid kind of Christoffel Symbols. Options are first or second.'

    if kind == 'second':
        coordinates = smp.ImmutableDenseNDimArray(coordinates)
        m = smp.Symbol('m') # Dummy Index for the sum
        n = metric.shape[0]
        g_inv = metric.inv()
        gammas = smp.MutableDenseNDimArray(smp.zeros(n**3), shape=(n,n,n))
        for k in range(n): #coordinates
            for i in range(n):
                for j in range(n):
                    gammas[k, i, j] = 0.5 * smp.Sum(g_inv[k,m]*(smp.diff(metric[j,m], coordinates[i]) + smp.diff(metric[m,i], coordinates[j]) - smp.diff(metric[i,j], coordinates[k])),
                                                    (m, 0, n-1)).doit()
        return smp.simplify(gammas)

    elif kind == 'first':
        coordinates = smp.ImmutableDenseNDimArray(coordinates)
        n = metric.shape[0]
        gammas = smp.MutableDenseNDimArray(smp.zeros(n**3), shape=(n,n,n))
        for c in range(n): #coordinates
            for a in range(n):
                for b in range(n):
                    gammas[c,a,b] = 0.5 * (smp.diff(metric[c,a], coordinates[b]) + smp.diff(metric[c,b], coordinates[a]) - smp.diff(metric[a,b], coordinates[c]))
        return smp.simplify(gammas)

#### Compute the Riemann Curvature Tensor

In local coordinates, the contravariant Riemann Curvature Tensor, $R_{\sigma \mu \nu}^{\rho}$, is given by: $$R_{\sigma \mu \nu}^{\rho} = \partial_{\mu} \Gamma_{\nu \sigma}^{\rho} - \partial_{\nu} \Gamma_{\mu \sigma}^{\rho} + \Gamma_{\mu \lambda}^{\rho} \Gamma_{\nu \sigma}^{\lambda} - \Gamma_{\nu \lambda}^{\rho} \Gamma_{\mu \sigma}^{\lambda}$$

It measures the failure of parallel transport along a vector field defined on the manifold. In order to lower the upper index to get the covariant Riemann tensor we have to multiply by the metric as follows: $$R_{\rho \sigma \mu \nu} = g_{\rho \lambda} R_{\sigma \mu \nu}^{\lambda}$$

In [90]:
def riemann_tensor(metric, coordinates, tensor_type='covariant'):
    assert tensor_type == 'covariant' or tensor_type == 'contravariant', 'Wrong tensor type. Options are contravariant and covariant'
    if tensor_type == 'covariant':
        gammas = christoffel_symbols(metric, coordinates)
        n = metric.shape[0]
        l = smp.Symbol('lambda') # Dummy Index for the sum
        k = smp.Symbol('k') # Dummy Index for the sum
        contr_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
        for rho in range(n):
            for sigma in range(n):
                for mu in range(n):
                    for nu in range(n):
                        contr_rm[rho, sigma, mu, nu] = smp.diff(gammas[rho, nu, sigma], coordinates[mu]) - smp.diff(gammas[rho, mu, sigma], coordinates[nu]) + smp.Sum(gammas[rho, mu, l] * gammas[l, nu, sigma], (l, 0, n-1)).doit() - smp.Sum(gammas[rho, nu, k] * gammas[k, mu, sigma], (k, 0, n-1)).doit()

        cov_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
        s = smp.Symbol('s') # Dummy Index for the sum
        for l in range(n):
            for i in range(n):
                for k in range(n):
                    for j in range(n):
                        cov_rm[l,i,k,j] = smp.Sum(metric[l,s]*contr_rm[s,i,k,j],(s, 0, n-1)).doit()
        return smp.simplify(cov_rm)

    elif tensor_type == 'contravariant':
        gammas = christoffel_symbols(metric, coordinates)
        n = metric.shape[0]
        l = smp.Symbol('lambda') # Dummy Index for the sum
        k = smp.Symbol('k') # Dummy Index for the sum
        contr_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
        for rho in range(n):
            for sigma in range(n):
                for mu in range(n):
                    for nu in range(n):
                        contr_rm[rho, sigma, mu, nu] = smp.diff(gammas[rho, nu, sigma], coordinates[mu]) - smp.diff(gammas[rho, mu, sigma], coordinates[nu]) + smp.Sum(gammas[rho, mu, l] * gammas[l, nu, sigma], (l, 0, n-1)).doit() - smp.Sum(gammas[rho, nu, k] * gammas[k, mu, sigma], (k, 0, n-1)).doit()

        return smp.simplify(contr_rm)

#### Compute the Ricci Tensor

The Ricci Curvature Tensor measures how a shape is deformed as we move along geodesics in the manifold. It is a symmetric bilinear form of rank 2. We get it by contracting the covariant Riemann Curvature Tensor by the metric: $$R_{ij} = g^{kl}R_{kilj}$$

In [91]:
def ricci_tensor(metric, coordinates):
    n = metric.shape[0]
    g_inv = metric.inv()
    rm = riemann_tensor(metric, coordinates, 'covariant')
    rc = smp.MutableDenseNDimArray(smp.zeros(n**2), shape=(n,n))
    a = smp.Symbol('a')
    b = smp.Symbol('b')
    for h in range(n):
        for k in range(n):
            rc[h,k] = smp.Sum(smp.Sum(g_inv[a,b]*rm[a,h,b,k] , (a, 0, n-1)), (b, 0, n-1)).doit()
    return smp.simplify(rc)

#### Compute Scalar Curvature

Finally, the scalar curvature is computed by further contracting the Ricci Tensor by the metric: $$\mathcal{R} = g^{ij}R_{ij}$$

In [92]:
def scalar_curvature(metric, coordinates):
    n = metric.shape[0]
    g_inv = metric.inv()
    rc = ricci_tensor(metric, coordinates)
    a = smp.Symbol('a')
    b = smp.Symbol('b')
    return smp.simplify(smp.Sum(smp.Sum(g_inv[a, b] * rc[a, b]  , (a, 0, n-1)), (b, 0, n-1)).doit())

#### Compute Gaussian Curvature

The Gaussian Curvature, K, is yet another intrinsic property of a manifold. It is defined as the product of the principal curvatures at any given point: $$K = k_{1}k_{2}$$ A more "Riemannian" definition for a 2-manifold, with respect to the metric, is: $$K = \frac{\langle(\nabla_{2} \nabla_{1} - \nabla_{1} \nabla_{2})e_{1}, e_{2}\rangle}{\det g}$$. An alternative definition, using the first and second fundamental forms, is: $$K = \frac{\det(II)}{\det(I)}$$ Gauss showed that the Gaussian Curvature is an intrinsic property and so it should only depend on the first fundamental form. Indeed, by the "Brioschi formula", we can express the Gaussian Curvature solely in terms of the coefficients of the first fundamental form: $$\displaystyle K = \frac{\begin{vmatrix}-\frac{1}{2}E_{vv} + F_{uv} - \frac{1}{2}G_{uu}&\frac{1}{2}E_{u}&F_{u} - \frac{1}{2}E_{v}\\ F_{v} - \frac{1}{2}G_{u}&E&F\\\frac{1}{2}G_{v}&F&G \end{vmatrix} - \begin{vmatrix}0&\frac{1}{2}E_{v}&\frac{1}{2}G_{u}\\ \frac{1}{2}E_{v}&E&F\\ \frac{1}{2}G_{u}&F&G \end{vmatrix}}{(EG - F^{2})^{2}}$$

In [93]:
def gaussian_curvature(metric, coordinates):
    E, F, G = first_fundamental_form(metric)
    left_mat = smp.Matrix([[-0.5*smp.diff(E, coordinates[1], coordinates[1]) + smp.diff(F, coordinates[0], coordinates[1]) -0.5*smp.diff(G, coordinates[0], coordinates[0]), 0.5*smp.diff(E, coordinates[0]), smp.diff(F, coordinates[0]) - 0.5*smp.diff(E, coordinates[1])], [smp.diff(F, coordinates[1]) - 0.5*smp.diff(G, coordinates[0]), E, F], [0.5*smp.diff(G, coordinates[1]), F, G]])

    right_mat = smp.Matrix([[0, 0.5*smp.diff(E, coordinates[1]), 0.5*smp.diff(G, coordinates[0])], [0.5*smp.diff(E, coordinates[1]), E, F], [0.5*smp.diff(G, coordinates[0]), F, G]])

    denominator = (E*G - F**2)**2

    K = (smp.det(left_mat) - smp.det(right_mat)) / denominator
    return smp.simplify(K)

#### Compute the Euler Characteristic

Given the Gaussian Curvature, K, of a 2-dimensional Riemannian manifold $\mathcal{M}$, by the Gauss-Bonnet theorem, the Euler Characteristic, $\chi$, is given by the following formula: $$\int_{\mathcal{M}} K dA + \int_{\partial \mathcal{M}} k_{g} ds = 2 \pi \chi$$ where $dA$ is the infinitesimal area element, $\partial \mathcal{M}$ the boundary of the manifold and $k_{g}$ the geodesic curvature of the boundary. If $\mathcal{M}$ is a manifold without boundary then the second integral of the left-hand side vanishes.

In [94]:
def euler_characteristic(metric, coordinates, boundary=None):
    if boundary is not None:
        print('No implementation for manifolds with boundary yet! Set boundary=None')
        return

    E, F, G = first_fundamental_form(metric)
    gauss_curvature = gaussian_curvature(metric, coordinates)
    dA = smp.sqrt(E * G - F**2)  # Don't forget to multiply by the area-element correction factor!
    integral = integrate(gauss_curvature * dA, (coordinates[0], 0, 2 * smp.pi), (coordinates[1], 0, smp.pi))
    return smp.simplify(integral / (2 * smp.pi))

#### Putting them all together in a class

In [95]:
class Intrinsic_Properties:
    def __init__(self, metric, coordinates):
        self.metric = metric
        self.coordinates = coordinates

    def first_fundamental_form(self):
        """
        :return: SymPy Expressions, the coefficients of the first fundamental form
        """
        E, F, G = self.metric.row(0)[0], self.metric.row(0)[1], self.metric.row(1)[1]
        return E, F, G


    def christoffel_symbols(self, kind='second'):
        """
        :param kind: (str): either first or second. For the following propertiew we only need the second kind
        :return: SymPy Expression, an nxnxn matrix
        """
        assert kind == 'first' or kind == 'second', 'Invalid kind of Christoffel Symbols. Options are first or second.'

        if kind == 'second':
            self.coordinates = smp.ImmutableDenseNDimArray(self.coordinates)
            m = smp.Symbol('m') # Dummy Index for the sum
            n = self.metric.shape[0]
            g_inv = self.metric.inv()
            gammas = smp.MutableDenseNDimArray(smp.zeros(n**3), shape=(n,n,n)) #Initialization
            for k in range(n): #coordinates
                for i in range(n):
                    for j in range(n):
                        gammas[k, i, j] = 0.5 * smp.Sum(g_inv[k,m]*(smp.diff(self.metric[j,m], self.coordinates[i]) + smp.diff(self.metric[m,i], self.coordinates[j]) - smp.diff(self.metric[i,j], self.coordinates[k])),
                                                        (m, 0, n-1)).doit()
            return smp.simplify(gammas)

        elif kind == 'first':
            self.coordinates = smp.ImmutableDenseNDimArray(self.coordinates)
            n = self.metric.shape[0]
            g_inv = self.metric.inv()
            gammas = smp.MutableDenseNDimArray(smp.zeros(n**3), shape=(n,n,n))
            for c in range(n): #coordinates
                for a in range(n):
                    for b in range(n):
                        gammas[c,a,b] = 0.5 * (smp.diff(self.metric[c,a], self.coordinates[b]) + smp.diff(self.metric[c,b], self.coordinates[a]) - smp.diff(self.metric[a,b], self.coordinates[c]))
            return smp.simplify(gammas)




    def riemann_tensor(self, tensor_type='covariant'):
        """
        :param tensor_type: (str): either covariant or contravariant. We only need the covariant type
        :return: SymPy Expression, an nxnxnxn matrix
        """
        assert tensor_type == 'covariant' or tensor_type == 'contravariant', 'Wrong tensor type. Options are contravariant and covariant'
        if tensor_type == 'covariant':
            gammas = self.christoffel_symbols(kind='second')
            n = self.metric.shape[0]
            l = smp.Symbol('lambda') # Dummy Index for the sum
            k = smp.Symbol('k') # Dummy Index for the sum
            contr_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
            for rho in range(n):
                for sigma in range(n):
                    for mu in range(n):
                        for nu in range(n):
                            contr_rm[rho, sigma, mu, nu] = smp.diff(gammas[rho, nu, sigma], self.coordinates[mu]) - smp.diff(gammas[rho, mu, sigma], self.coordinates[nu]) + smp.Sum(gammas[rho, mu, l] * gammas[l, nu, sigma], (l, 0, n-1)).doit() - smp.Sum(gammas[rho, nu, k] * gammas[k, mu, sigma], (k, 0, n-1)).doit()

            cov_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
            s = smp.Symbol('s') # Dummy Index for the sum
            for l in range(n):
                for i in range(n):
                    for k in range(n):
                        for j in range(n):
                            cov_rm[l,i,k,j] = smp.Sum(self.metric[l,s]*contr_rm[s,i,k,j],(s, 0, n-1)).doit()
            return smp.simplify(cov_rm)

        elif type == 'contravariant':
            gammas = self.christoffel_symbols(kind='second')
            n = self.metric.shape[0]
            l = smp.Symbol('lambda') # Dummy Index for the sum
            k = smp.Symbol('k') # Dummy Index for the sum
            contr_rm = smp.MutableDenseNDimArray(smp.zeros(n**4), shape=(n,n,n,n))
            for rho in range(n):
                for sigma in range(n):
                    for mu in range(n):
                        for nu in range(n):
                            contr_rm[rho, sigma, mu, nu] = smp.diff(gammas[rho, nu, sigma], self.coordinates[mu]) - smp.diff(gammas[rho, mu, sigma], self.coordinates[nu]) + smp.Sum(gammas[rho, mu, l] * gammas[l, nu, sigma], (l, 0, n-1)).doit() - smp.Sum(gammas[rho, nu, k] * gammas[k, mu, sigma], (k, 0, n-1)).doit()
            return contr_rm


    def ricci_tensor(self):
        """
        :return:  SymPy Expression, an nxn matrix
        """
        n = self.metric.shape[0]
        g_inv = self.metric.inv()
        rm = self.riemann_tensor('covariant')
        rc = smp.MutableDenseNDimArray(smp.zeros(n**2), shape=(n,n))
        a = smp.Symbol('a')
        b = smp.Symbol('b')
        for h in range(n):
            for k in range(n):
                rc[h,k] = smp.Sum(smp.Sum(g_inv[a,b]*rm[a,h,b,k] , (a, 0, n-1)), (b, 0, n-1)).doit()
        return smp.simplify(rc)


    def scalar_curvature(self):
        """
        :return:  SymPy Expression, the Scalar Curvature (a real number)
        """
        n = self.metric.shape[0]
        g_inv = self.metric.inv()
        rc = self.ricci_tensor()
        a = smp.Symbol('a')
        b = smp.Symbol('b')
        return smp.simplify(smp.Sum(smp.Sum(g_inv[a, b] * rc[a, b]  , (a, 0, n-1)), (b, 0, n-1)).doit())


    def gaussian_curvature(self):
        """
        :return:  SymPy Expression, the Gaussian Curvature
        """
        E, F, G = self.first_fundamental_form()
        left_mat = smp.Matrix([[-0.5*smp.diff(E, self.coordinates[1], self.coordinates[1]) + smp.diff(F, self.coordinates[0], self.coordinates[1]) -0.5*smp.diff(G, self.coordinates[0], self.coordinates[0]), 0.5*smp.diff(E, self.coordinates[0]), smp.diff(F, self.coordinates[0]) - 0.5*smp.diff(E, self.coordinates[1])], [smp.diff(F, self.coordinates[1]) - 0.5*smp.diff(G, self.coordinates[0]), E, F], [0.5*smp.diff(G, self.coordinates[1]), F, G]])

        right_mat = smp.Matrix([[0, 0.5*smp.diff(E, self.coordinates[1]), 0.5*smp.diff(G, self.coordinates[0])], [0.5*smp.diff(E, self.coordinates[1]), E, F], [0.5*smp.diff(G, self.coordinates[0]), F, G]])

        denominator = (E*G - F**2)**2

        K = (smp.det(left_mat) - smp.det(right_mat)) / denominator
        return smp.simplify(K)

    def euler_characteristic(self, coordinate_restrictions, boundary=None):
        """
        :param coordinate_restrictions: list with the boundaries of integration. For example for the 2-sphere, 0 <= θ <= 2π and 0 <=φ < π
        :param boundary: Need to know the boundary of the manifold as well as how to compute the geodesic curvature in local coordinates
        :return: SymPy Expression, the Euler Characteristic χ (an integer)
        """
        if boundary is not None:
            print('No implementation for manifolds with boundary yet! Set boundary=None')
            return

        E, F, G = self.first_fundamental_form()
        gauss_curvature = self.gaussian_curvature()
        dA = smp.sqrt(E * G - F**2)  # Don't forget to multiply by the area-element correction factor!
        integral = integrate(gauss_curvature * dA, coordinate_restrictions[0], coordinate_restrictions[1])
        return smp.simplify(integral / (2 * smp.pi))

#### Example: 2-Sphere

In [96]:
#Standard metric of the 2-sphere in spherical coordinates (theta, phi)
R = smp.Symbol('R')
theta = smp.Symbol('theta')
phi = smp.Symbol('phi')
sphere_local_coordinates = [theta, phi]
sphere_g = smp.Matrix([[R**2, 0], [0, R**2 * (smp.sin(theta))**2]])
sphere_intr_prop = Intrinsic_Properties(metric=sphere_g, coordinates=sphere_local_coordinates)

In [97]:
sphere_g

Matrix([
[R**2,                  0],
[   0, R**2*sin(theta)**2]])

In [98]:
sphere_intr_prop.christoffel_symbols(kind='second')

[[[0, 0], [0, -0.5*sin(2*theta)]], [[0, 1.0/tan(theta)], [1.0/tan(theta), 0]]]

In [99]:
sphere_intr_prop.riemann_tensor(tensor_type='covariant')

[[[[0, 0], [0, 0]], [[0, R**2*(0.5*sin(2*theta)/tan(theta) - 1.0*cos(2*theta))], [R**2*(-0.5*sin(2*theta)/tan(theta) + 1.0*cos(2*theta)), 0]]], [[[0, -1.0*R**2*sin(theta)**2], [1.0*R**2*sin(theta)**2, 0]], [[0, 0], [0, 0]]]]

In [100]:
sphere_intr_prop.ricci_tensor()

[[1.0, 0], [0, 0.5*sin(2*theta)/tan(theta) - 1.0*cos(2*theta)]]

In [101]:
sphere_intr_prop.scalar_curvature()

2.0/R**2

In [102]:
sphere_intr_prop.gaussian_curvature()

1.0/R**2

In [103]:
sphere_intr_prop.euler_characteristic([(sphere_local_coordinates[0], 0, 2*smp.pi), (sphere_local_coordinates[1], 0, smp.pi)])

2.0*sqrt(R**4)/R**2

#### Example: 2-Torus

In [104]:
# Metric of the torus
R, r = smp.symbols('R r') # The big and small radius
theta, phi = smp.symbols('theta phi') # Coordinates (both between 0 and 2π)
torus_local_coordinates = [theta, phi]
torus_g = smp.Matrix([[(R + r * smp.cos(phi))**2, 0], [0, r**2]])
torus_intr_prop = Intrinsic_Properties(metric=torus_g, coordinates=torus_local_coordinates)

In [105]:
torus_g

Matrix([
[(R + r*cos(phi))**2,    0],
[                  0, r**2]])

In [106]:
torus_intr_prop.christoffel_symbols(kind='second')

[[[0, -1.0*r*sin(phi)/(R + r*cos(phi))], [-1.0*r*sin(phi)/(R + r*cos(phi)), 0]], [[1.0*(R + r*cos(phi))*sin(phi)/r, 0], [0, 0]]]

In [107]:
torus_intr_prop.riemann_tensor(tensor_type='covariant')

[[[[0, 0], [0, 0]], [[0, 1.0*r*(R + r*cos(phi))*cos(phi)], [-1.0*r*(R + r*cos(phi))*cos(phi), 0]]], [[[0, -1.0*r*(R + r*cos(phi))*cos(phi)], [1.0*r*(R + r*cos(phi))*cos(phi), 0]], [[0, 0], [0, 0]]]]

In [108]:
torus_intr_prop.ricci_tensor()

[[1.0*(R + r*cos(phi))*cos(phi)/r, 0], [0, 1.0*r*cos(phi)/(R + r*cos(phi))]]

In [109]:
torus_intr_prop.scalar_curvature()

2.0*cos(phi)/(r*(R + r*cos(phi)))

In [110]:
torus_intr_prop.gaussian_curvature()

1.0*cos(phi)/(r*(R + r*cos(phi)))

In [111]:
torus_intr_prop.euler_characteristic(coordinate_restrictions=[(torus_local_coordinates[0], 0, 2*smp.pi), (torus_local_coordinates[1], 0, 2*smp.pi)])

1.0*Integral(sqrt(r**2*(R + r*cos(phi))**2)*cos(phi)/(R + r*cos(phi)), (phi, 0, 2*pi))/r

The above integral is indeed 0 but sympy does not reach the final result for some reason